# <b>*Easy DAIN* for Colab</b>

Based on the original colab notebook. This will install DAIN in your Google Drive (~300MB)
<br>You need to build it once, it takes around 25 minutes. You can find the installer at the bottom of the notebook.
<br>Video processing time depends on video length and resolution. Max video resolution is 720p
<br>I've also added a neat setup feature, where instead of waiting 25 minutes for each runtime, it will save the built packages in your Drive and then import them, it's a lot faster.
<br>• Tip: Set the Hardware accelerator to GPU


## Manual

*Setting up DAIN:*

1.   If this is your first time opening this notebook, please read it's description up there.
2.   Run `Setup`, connect to your Drive and install dependences on this machine.
3.   Go to `Installer/Updater` and click run, this will build and save DAIN and it's packages in your Drive. You only need to do it once.
4.   Once built, repeat step **2** every time you connect to a new runtime.

*Running DAIN:*


*   Set the video *input* and *output* path, relative to your Drive root "*My Drive"*
*   Set the output *target framerate* interpolated by DAIN, make sure it's lower than the half of the output framerate. Example: input: 30fps, output: 60fps.
*   For making animated pictures, there's the *Looping* option you can tick, this will use the first frame of the video at the end so it can transition back, closing the loop.
*   If the input has *Audio*, you can tick the checkbox, else don't.
*   When everything is set, run the cell and keep the page alive until processing ends.


*Extra information:*
*   Depending on the video length, resolution and interpolation ratio, the processing time will be long, make sure you're connected to the machine all the way to the end, or at least keeping it active before it shuts down.
*  DAIN lowers the resolution of the video a bit due to deformations in the borders caused by the interpolation process, so it crops it. Is not so noticeable but you can disable/enable it ticking the Resize Hotfix checkbox.
*  If you're using normal Colab (not pro) the runtime time limit you have is **5 hours**, so make sure to split the video into parts or else everything will stop. 
Here's a table of max video input length:
  *  720p: 4 minutes at 30fps
  *  480p: 5.5 minutes at 30fps

  Tested on a Tesla T4 GPU, but there's faster ones. You can get GPU info at `Video Tools`

**PLEASE NOTE:** The output video speed will be slower than the original, there is a thing i didn't quite understand, it may be the time step or time between frames, but if you have a video editor, speed up the footage to match the original one, use the audio channel separatedly if it has one.





In [ ]:
%%javascript
//#@markdown For Colab pro users: [open this]
/*Sometimes if the processing time is long, the runtime will disconnect, even still running.
 If nobody's present for clicking reconnect, it will reset and stop all processing.
 Here's a little code that will help you with that :]
 Open developer-settings (Ctrl + Shift + I) or Options > Tools > Developer Settings,
 Go to Console
 Copy the following code:*/
function ConnectButton(){
    console.log("Connect pushed"); 
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click();
    //Click close dialog button
    setTimeout(function(){
      try{
      document.querySelector("colab-dialog").querySelector("paper-dialog").querySelector("colab-sessions-dialog").shadowRoot.getElementById("footer").querySelector("div").getElementsByClassName("dismiss")[0].click();
      }catch(e){};
    },1000);
}
clearInterval(window.timer);
timer = setInterval(ConnectButton,60000);
/*This will click 'connect' every minute, so it can keep the runtime alive.
 If you want to stop it, type*/ clearInterval(timer) /*in the console.
 Hope it helps!*/

# EZ DAIN




## Setup

In [ ]:
#@markdown <b>Connect to your Google Drive<b>
# Go to the link
# Copy the code and paste it here.
from google.colab import drive
drive.mount('/content/gdrive')
print('Google Drive connected.')

In [ ]:
#@markdown <b>Setup DAIN</b><br>Run this setup once per runtime.
#@markdown Set folder where DAIN will be or is already installed in your Google Drive.
DRIVE_PATH = '/content/gdrive/MyDrive'
DRIVE_FOLDER = '/content/gdrive/MyDrive'
DAIN_PATH ="/DAIN"#@param{type:"string"}
DAIN_FOLDER = DRIVE_PATH+DAIN_PATH
DAIN_FOLDERCMD = DRIVE_FOLDER+DAIN_PATH

dont_install_dependences = False#@param{type:'boolean'}

import sys
import subprocess
import os

# YESNO questionbox
def yn(question):
    while "the answer is invalid":
        reply = str(input(question+' (y | n): ')).lower().strip()
        if reply[:1] == 'y':
            return True
        else:
            return False
# READFILE
def readfile(path):
  try:
   return str(subprocess.check_output(['head', path])).replace("\\n'",'').replace("b'",'')
  except:
   return "exit 1"

# WRITEPACKAGES
def copypackages(fromf,to):
  for pkg in DAINPACKAGES:
    print('Copying package: '+pkg)
    %cp -f -r {fromf}{pkg} {to}

PY = str(sys.version_info.major) + '.' + str(sys.version_info.minor)
PYTHON_DIR = "/usr/local/lib/python"+PY

# Get package python version where were compiled
PKGPYVER = readfile(DAIN_FOLDER+"/dain-packages/pypkg-version.txt")

if PKGPYVER != 'exit 1':
  if PKGPYVER != PY:
    print("\033[93mWarning: Packages version ("+PKGPYVER+") may not work with current Python version ("+PY+")\nIf frame interpolation fails, please update DAIN packages with the updater at the bottom of this notebook\033[0m")
    PYV = PKGPYVER
  else:
    PYV = PY
else: PYV = PY

# List of packages built by DAIN, build them once, save, and restore them any time
DAINPACKAGES=[
"correlation_cuda-0.0.0-py"+PYV+"-linux-x86_64.egg",
"depthflowprojection_cuda-0.0.0-py"+PYV+"-linux-x86_64.egg",
"easy-install.pth", 
"filterinterpolation_cuda-0.0.0-py"+PYV+"-linux-x86_64.egg",
"flowprojection_cuda-0.0.0-py"+PYV+"-linux-x86_64.egg",
"interpolationch_cuda-0.0.0-py"+PYV+"-linux-x86_64.egg",
"interpolation_cuda-0.0.0-py"+PYV+"-linux-x86_64.egg",
"mindepthflowprojection_cuda-0.0.0-py"+PYV+"-linux-x86_64.egg",
"separableconv_cuda-0.0.0-py"+PYV+"-linux-x86_64.egg",
"separableconvflow_cuda-0.0.0-py"+PYV+"-linux-x86_64.egg"]

# Check VRAM
VRAM = !nvidia-smi --query-gpu=memory.total --format=csv
VRAM = VRAM[1].replace(' MiB','')
if VRAM!='': 
  VRAM = int(VRAM) 
else: 
  VRAM = 0

if VRAM == 0:
  raise Exception("\033[93mNo GPU was detected in this runtime, DAIN won't work. Set Environment runtime type to GPU.")


## Insalling Dependences
# Install known used versions of PyTorch and SciPy
if not dont_install_dependences:
  !pip install torch==1.4.0+cu100 torchvision==0.5.0+cu100 -f https://download.pytorch.org/whl/torch_stable.html
  !pip install scipy==1.1.0

  !CUDA_VISIBLE_DEVICES=0
  !sudo apt-get install imagemagick imagemagick-doc
  print("Finished installing dependencies.")

# Restoring packages
if os.path.isdir(DAIN_FOLDER):
  copypackages(DAIN_FOLDERCMD+'/dain-packages/',PYTHON_DIR+'/dist-packages')
print('\033[92mAll Done.')

## App

In [ ]:

# YOUR VIDEO FILE PATH (From GDrive)
INPUT_FILEPATH = "/input.mp4" #@param{type:"string"}
 
# THE OUTPUT PATH
OUTPUT_FILE_PATH = "/output.mp4"#@param{type:"string"}
 
# TARGET FRAME RATE (FINAL RESULT FPS)
TARGET_FPS =  60#@param{type:"number"}
 
# IF YOUR VIDEO IS LOOPING, SET TO TRUE
#markdown If your video is looping (for gifs, etc) check here:
LOOPING = False #@param{type:"boolean"}
#markdown If the video has audio, check here, else don't!
AUDIO = False #@param{type:"boolean"}
 
# SOME EXTRA OPTIONS
# VIDEO SCALE FIX (Read DAIN papers)
#markdown Extra options:
RESIZE_HOTFIX = True#@param{type:'boolean'}

import os
import cv2
from IPython.display import clear_output
from IPython.display import display
import shutil
import subprocess as sp
import numpy as np
import sys
# Check your current GPU
# If you are lucky, you get 16GB VRAM. If you are not lucky, you get less. VRAM is important. The more VRAM, the higher the maximum resolution will go.
 
# 16GB: Can handle 720p. 1080p will procude an out-of-memory error. 
# 8GB: Can handle 480p. 720p will produce an out-of-memory error.
print("|>\033[92mSYSTEM: ---GPU info---\033[0m")
!nvidia-smi --query-gpu=gpu_name,driver_version,memory.total --format=csv

try:
  VRAM
except:
  raise Exception("\033[93mPlease run DAIN Setup first.")

if VRAM == 0:
  raise Exception("\033[93mNo GPU was detected in this runtime, DAIN won't work. Set Environment runtime type to GPU.")

%cd /root
%shell mkdir -p '/content/DAIN'
 
# Detecting FPS of input file.
print("|>\033[92mCV2: Detect input video fps...\033[0m")
%shell yes | cp -f /content/gdrive/MyDrive/{INPUT_FILEPATH} /content/DAIN
filename = os.path.basename(INPUT_FILEPATH)
cap = cv2.VideoCapture(f'/content/DAIN/{filename}')
fps = cap.get(cv2.CAP_PROP_FPS)
# Video resolution
fh = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fw = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))

if (fh >= 720 or fw >= 720) and VRAM < 10000:
  raise ValueError('"\033[93mWarning: Video resolution ('+str(fw)+'x'+str(fh)+') is too high for VRAM ('+str(VRAM)+'GB). Try downscaling the video with the Video Tools or connect to a better runtime.')
if (fh >= 1080 and fw >= 1080):
   raise ValueError("\033[93mWarning: Video resolution ("+str(fw)+'x'+str(fh)+") is not supported")

if (fps/TARGET_FPS>0.5):
  print("Define a higher fps, because there is not enough time for new frames. (Old FPS)/(New FPS) should be lower than 0.5. Interpolation will fail if you try.")
  raise SystemExit
 
# ffmpeg extract - Generating individual frame PNGs from the source file.
print("|>\033[92mFFMPEG: Getting frames...\033[0m")
%shell rm -rf '/content/DAIN/frames'
%shell mkdir -p '/content/DAIN/frames'
 
%shell ffmpeg -i '/content/DAIN/{filename}' '/content/DAIN/frames/%05d.png'
png_generated_count_command_result = %shell ls '/content/DAIN/frames' | wc -l

clear_output()
 
pngs_generated_count = int(png_generated_count_command_result.output.strip())

if LOOPING==True:
  pngs_generated_count += 1
  original = "/content/DAIN/frames/00001.png"
  target = "/content/DAIN/frames/"+str(pngs_generated_count).zfill(5)+".png"
  shutil.copyfile(original, target)

# Log got cleared
print("|>\033[92mSYSTEM: ---GPU info---\033[0m")
!nvidia-smi --query-gpu=gpu_name,driver_version,memory.total --format=csv

print(f"Input FPS: {fps}")
print(f"{pngs_generated_count} frame PNGs generated.")
 
# Checking if PNG do have alpha
print("|>\033[92m[SUBPROCESS]IDENTIFY: Checking image alpha...\033[0m")

%cd /content/DAIN/frames
channels = sp.getoutput('identify -format %[channels] 00001.png')
print (f"{channels} detected")
 
# Removing alpha if detected
if "a" in channels:
  print("Alpha detected and will be removed.")
  print(sp.getoutput('find . -name "*.png" -exec convert "{}" -alpha off PNG24:"{}" \;'))
 
# Interpolation
print("|>\033[92mDAIN: Frame interpolation...\033[0m")
%shell mkdir -p '/content/DAIN/frames'
%cd {DAIN_FOLDERCMD}

!python -W ignore colab_interpolate.py --netName DAIN_slowmotion --time_step {fps/TARGET_FPS} --start_frame 1 --end_frame {pngs_generated_count} --frame_input_dir '/content/DAIN/frames' --frame_output_dir '/content/DAIN/frames'

# Finding DAIN Frames, upscaling and cropping to match original
print("|>\033[92mNUMPY: Upscaling and cropping...\033[0m")

%cd /content/DAIN/frames
dh = display('',display_id=True)
if (RESIZE_HOTFIX==True):
  images = []
  for imagename in os.listdir(f'/content/DAIN/frames'):
    img = cv2.imread(os.path.join(f'/content/DAIN/frames',imagename))
    part_filename = os.path.splitext(imagename)
    if(part_filename[0].endswith('0')==False):
      dh.update("Processing frame "+imagename)
      dimension = (img.shape[1]+2, img.shape[0]+2)
      resized = cv2.resize(img, dimension, interpolation=cv2.INTER_LANCZOS4)
      crop = resized[1:(dimension[1]-1), 1:(dimension[0]-1)]
      cv2.imwrite(part_filename[0]+".png", crop)
del dh
# Create video
print("|>\033[92mFFMPEG: Joining frames to make a video...\033[0m")

AUDIOCMD = ""

if AUDIO:
  %cd /content/DAIN/frames
  %shell ffmpeg -i '/content/DAIN/{filename}' -acodec copy output-audio.aac
  AUDIOCMD = '-itsoffset 0.770 -i output-audio.aac '

%shell ffmpeg -framerate {TARGET_FPS} -f image2 -pattern_type glob -i '*.png' {AUDIOCMD}-pix_fmt yuv420p -y "/content/gdrive/MyDrive{OUTPUT_FILE_PATH}"

print("|>\033[92mSYSTEM: Cleaning...\033[0m")
%shell rm -rf /content/DAIN/frames/
%shell rm -rf "/content/DAIN/{filename}"

print("|>\033[92mEZDAIN: All done!\033[0m")

## Installer/Updater

In [ ]:
#@markdown <b>Install/Update<br>
#@markdown (Run it once, and every time python updates for compilaton reasons)</b>

# Clear DAIN folder if installed
DAIN_INSTALLED = os.path.isdir(DAIN_FOLDER)

if DAIN_INSTALLED:
    rep = yn('DAIN is already installed in your Drive, uninstall it?')
    if rep:
      !rm rf {DAIN_FOLDERCMD}
    else:
      print('Installation cancelled, exiting...')
      raise SystemExit
# Clone DAIN sources
%cd /content
!git clone -b master --depth 1 https://github.com/baowenbo/DAIN {DAIN_FOLDERCMD}
%cd {DAIN_FOLDERCMD}
!git log -1

# This takes a while. Just wait. ~15 minutes.
# Building DAIN.
%cd {DAIN_FOLDERCMD}/my_package/
%shell chmod u+x ./build.sh
!./build.sh
print("Building #1 done.")

# Wait again. ~5 minutes.
# Building DAIN PyTorch correlation package.
%cd {DAIN_FOLDERCMD}/PWCNet/correlation_package_pytorch1_0
%shell chmod u+x ./build.sh
!./build.sh
print("Building #2 done.")

# Downloading pre-trained model
%cd {DAIN_FOLDERCMD}
!mkdir model_weights
!wget -O model_weights/best.pth http://vllab1.ucmerced.edu/~wenbobao/DAIN/best.pth

# Copy created packages to dain folder
!mkdir dain-packages
copypackages(PYTHON_DIR+'/dist-packages/',DAIN_FOLDER+'/dain-packages')
# Save python version where those packages were compiled
%shell echo {PY} > {DAIN_FOLDERCMD+'/dain-packages/pypkg-version.txt'}
print("Done")

# Video Tools
Little set of options to edit your video before or after processing. Connect to your Drive in DAIN Setup first.

In [ ]:
DRIVE_FOLDER = '/content/gdrive/MyDrive'
SOURCE_INPUT = "/input.mp4"#@param{type:'string'}
CLIP_OUTPUT = "/output.mp4"#@param{type:'string'}
CLIP_START = '0:00:00.00'#@param{type:'string'}
CLIP_END = '0'#@param{type:'string'}
RESIZE = "don't resize" #@param ["don't resize", "720p", "480p"]
#@markdown Required for resize:
ORIENTATION = "landscape" #@param ["landscape", "portrait"]
#@markdown (Set to 0 for auto detect, format hh:mm:ss:ms)
INPUT = DRIVE_FOLDER+SOURCE_INPUT
OUTPUT = DRIVE_FOLDER + CLIP_OUTPUT

import subprocess
dur = subprocess.getoutput("ffprobe "+INPUT+" 2>&1 | grep 'Duration:' ")
dur = dur[12:dur.index(",")]

if CLIP_START == '0':
  CLIP_START = '0:00:00.00'
  
if CLIP_END == '0':
  CLIP_END = dur

if RESIZE != "don't resize":
  if ORIENTATION == 'landscape':
    RESIZE = '-vf "scale=-1:'+RESIZE.replace('p','')+'" '
  else:
    RESIZE = '-vf "scale='+RESIZE.replace('p','')+':-1" '
else:
  RESIZE = ''

%shell ffmpeg -i {INPUT} -c copy -ss {CLIP_START} -t {CLIP_END} {RESIZE}-y {OUTPUT}

In [ ]:
#@markdown Query GPU info
!nvidia-smi